modified by Adrain (21/05/28)

## 9-5.py

이 부분은 정상 작동 -> 필요함

In [ ]:
import numpy as np
import random
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque

# 하이퍼 매개변수 설정
rho=0.9 # 학습률
lamda=0.99 # 할인율
eps=0.9
eps_decay=0.999
batch_siz=64
n_episode=100

# 신경망을 설계해주는 함수
def deep_network():
    mlp=Sequential()
    mlp.add(Dense(32,input_dim=env.observation_space.shape[0],activation='relu'))
    mlp.add(Dense(32,activation='relu'))
    mlp.add(Dense(env.action_space.n,activation='linear'))
    mlp.compile(loss='mse',optimizer='Adam')
    return mlp

# DQN 학습
def model_learning():
    mini_batch=np.asarray(random.sample(D,batch_siz))
    state=np.asarray([mini_batch[i,0] for i in range(batch_siz)])
    action=mini_batch[:,1]
    reward=mini_batch[:,2]
    state1=np.asarray([mini_batch[i,3] for i in range(batch_siz)])
    done=mini_batch[:,4]

    target=model.predict(state)
    target1=model.predict(state1)

    for i in range(batch_siz):
        if done[i]:
            target[i][action[i]]=reward[i]
        else:
            target[i][action[i]]+=rho*((reward[i]+lamda*np.amax(target1[i]))-target[i][action[i]]) # Q 러닝(식 (9.19))
    model.fit(state,target,batch_size=batch_siz,epochs=1,verbose=0)

env=gym.make("CartPole-v0")

model=deep_network() # 신경망 생성
D=deque(maxlen=2000) # 리플레이 메모리 초기화
scores=[]
max_steps=env.spec.max_episode_steps

# 신경망 학습
for i in range(n_episode):
    s=env.reset()
    long_reward=0

    while True:
        r=np.random.random()
        eps=max(0.01,eps*eps_decay) # 엡시론을 조금씩 줄여나감
        if(r<eps):
            a=np.random.randint(0,env.action_space.n) # 랜덤 정책
        else:
            q=model.predict(np.reshape(s,[1,4])) # 신경망이 예측한 행동
            a=np.argmax(q[0])
        s1,r,done,_=env.step(a)
        if done and long_reward<max_steps-1: # 실패
            r=-100

        D.append((s,a,r,s1,done))

        if len(D)>batch_siz*3:
            model_learning()

        s=s1
        long_reward+=r
        
        if done:
            long_reward=long_reward if long_reward==max_steps else long_reward+100
            print(i,"번째 에피소드의 점수:",long_reward)
            scores.append(long_reward)
            break

    if i>10 and np.mean(scores[-5:])>(0.95*max_steps):
        break
    
# 신경망 저장
model.save("./model/cartpole_by_DQN.h5")
env.close()

import matplotlib.pyplot as plt

plt.plot(range(1,len(scores)+1),scores)
plt.title('DQN scores for CartPole-v0')
plt.ylabel('Score')
plt.xlabel('Episode')
plt.grid()
plt.show()

## 9-6.py

오류가 나는 이유:
**GYM 모듈은 화면에서 GL(Graphics Library)을 사용하여 디스플레이를 렌더링**
- 그러나 Colab은 웹에서 노트북으로 실행되므로 화면에 직접 표시 할 수 없습니다. HTML을 통해서만 결과를 표시 할 수 있습니다.

원문:
https://www.debugcn.com/ko/article/1105981.html

**해결책: pyvirtualdisplay(python 가상 디스플레이 모듈을 사용)**

### SETTING

1. 필요한 모듈 설치

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules # 구글 코랩에서 실행되는지 여부
IS_KAGGLE = "kaggle_secrets" in sys.modules # 캐글에서 실행되는지 여부

if IS_COLAB or IS_KAGGLE: # 코랩 or 캐글에서 동작하는 경우 필요한 것들 설치
    !apt update && apt install -y libpq-dev libsdl2-dev swig xorg-dev xvfb
    !pip install -q -U tf-agents pyvirtualdisplay gym[atari,box2d] # 이번 에러의 해결책의 핵심 1)
    # pyvirtualdisplay를 설치한다. 
    #(tf-agents와 사용될 gym[atari,box2d]도 같이 설치됨 -> 기존에 있던 코드에서 가져왔기 때문에 있는 코드)

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'): # 대충 GPU 사용하라는 뜻이지만 CPU로 해도 무방할...듯?
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:13 http://ppa.launchpad.net/cran/

필요한 모듈 import

In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Common imports
import numpy as np
import os

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml') # javascript html 애니메이션 설정

그림 저장용 코드

In [3]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rl"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300): # 출력을 이미지로 저장하는 기능
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

def plot_environment(env, figsize=(5,4)): # 화면에 띄우는 기능
    plt.figure(figsize=figsize)
    img = env.render(mode="rgb_array")
    plt.imshow(img)
    plt.axis("off")
    return img

2. 중요한 파이썬 가상 디스플레이 import

제대로 디스플레이가 import되어있는지 확인하려면 print(display)를 했을 때 IPython이 아닌 pyvirtualdisplay가 떠야 정상

In [4]:
try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

In [5]:
print(display) # -> 정상적으로 디스플레이 설정이 되었음을 알수 있는 부분

동작을 애니메이션으로 표현하는 부분

In [6]:
def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = animation.FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

### 기존 코드

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
import numpy as np
import gym
import time

# 신경망 블러옴
model=load_model('./model/cartpole_by_DQN.h5')

env=gym.make("CartPole-v0")
long_reward=0

# CartPole 플레이
s=env.reset()

frames = []
while True:
    q=model.predict(np.reshape(s,[1,4])) # 신경망이 예측한 행동
    a=np.argmax(q[0])
    s1,r,done,_=env.step(a)
    s=s1
    long_reward+=r
    
# ============ 기존 코드 ===========
#     env.render() # 이 부분에서 앞의 setting을 해주지 않으면 colab에서는 base에러가 난다
#     env.render()의 출력값은 True, None or Error인듯 하니 원하는 그림이 나오지 않는다.

# ============ 그림 출력하는 것 ============
#     plot_environment(env) => 사진을 1개씩 출력을 하게 된다.
#     plt.show()

# ============ 애니메이션 추가를 위한 frame화 시키기 ============
    img = env.render(mode="rgb_array")
    frames.append(img)
    time.sleep(0.02) # 동작을 위한 딜레이

    if done:
        print("에피소드의 점수:",long_reward)
        break

env.close()

에피소드의 점수: 200.0


학습 결과 영상으로 보기

In [8]:
plot_animation(frames)